# Gradient Boost Tree Algorithm

#### Importación de librerías ⬇️

In [1]:
import pickle

In [2]:
# Importar data general
with open('model.pkl', 'rb') as f:
    data = pickle.load(f)

# Importar data de las 10 enfermedades más comunes
with open('model1.pkl', 'rb') as f:
    data2 = pickle.load(f)
    
print(data)
print(data2)

           AÑO   MES  DIASESTANCIA  SEXO  PPERTENENCIA  EDAD  PERIODOEDA  \
117659  2012.0   8.0           3.0   2.0           9.0  31.0         3.0   
56693   2020.0   5.0           2.0   1.0           9.0   5.0         3.0   
228995  2020.0   7.0           1.0   2.0           9.0  57.0         3.0   
49633   2017.0  12.0           2.0   1.0           4.0  37.0         3.0   
111288  2013.0   7.0           2.0   1.0           6.0  70.0         3.0   
...        ...   ...           ...   ...           ...   ...         ...   
180642  2018.0  11.0           4.0   1.0           4.0  13.0         3.0   
136546  2021.0   1.0           1.0   1.0           4.0  35.0         3.0   
198495  2021.0   1.0           1.0   2.0           4.0  24.0         3.0   
47412   2012.0   5.0           1.0   2.0           4.0   6.0         3.0   
191508  2019.0   3.0           1.0   2.0           9.0  41.0         3.0   

        DEPTORESIDEN  MUNIRESIDEN  CAUFIN  TRATARECIB  CLUSTER  
117659           1.0  

/var/folders/1_/typt0bhn7_q_v0p0c8csdvmh0000gn/T/ipykernel_6726/530943450.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  data = pickle.load(f)
